In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.firefox.options import Options
from pathlib import Path
import pandas as pd
import json

# Constants

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
PATH_DATA = Path.cwd().parent.parent.parent / 'data'
PATH_DATA.exists()

True

In [5]:
PATH_PREPROCESSED = PATH_DATA / 'preprocessed'
PATH_PREPROCESSED.exists()

True

In [6]:
PATH_DF_URLS_ARCHAEA = PATH_PREPROCESSED / 'archaea' / 'archaea_name_formatting.csv'
PATH_DF_URLS_ARCHAEA.exists()

True

In [7]:
PATH_RESULTS = PATH_DATA / 'results'
PATH_RESULTS.exists()

True

In [8]:
PATH_RESULTS_LIFESTYLE_ARCHAEA = PATH_RESULTS / 'lifestyle' / 'archaea'
PATH_RESULTS_LIFESTYLE_ARCHAEA.exists()

True

In [9]:
PATH_DESTINY_LIFESTYLE_ARCHAEA = PATH_RESULTS_LIFESTYLE_ARCHAEA / 'lifestyle_archaea.json'
PATH_DESTINY_LIFESTYLE_ARCHAEA.exists()

True

In [10]:
PATH_DF_URLS_BACTERIA = PATH_PREPROCESSED / 'bacteria' / 'bacteria_name_formatting.csv'
PATH_DF_URLS_BACTERIA.exists()

True

# webdriver

In [9]:
driver = webdriver.Firefox()

# Capture env data

In [10]:
driver.implicitly_wait(180)
driver.get("https://gcm.wdcm.org/search?search=Acidianus%20brierleyi&list=isolate")
table_rows = driver.find_elements("css selector", "div.fd-isolate table tr")

## Don't wait for the page to charge completely

In [11]:
# options = Options()
# options.page_load_strategy = 'none'

In [12]:
# driver = webdriver.Firefox(options=options)

In [13]:
# driver.get("https://gcm.wdcm.org/search?search=Acidianus%20brierleyi&list=isolate")
# try:
#     WebDriverWait(driver, 35).until(
#         EC.presence_of_element_located(
#             ('css selector', 'div.fd-isolate table')
#         )
#     )
#     print("Table found")
# except NoSuchElementException as nsee:
#     print("Failed to find the table")
#     print(nsee)
# except Exception as e:
#     print("Didn't find the table")
#     print(e)


In [14]:
print(len(table_rows[0].text.split("\n")))
print(table_rows[0].text.split("\n"))

8
['Algae', 'Archaea', 'Bacterium', 'Fungi', 'Protozoa', 'Virus', 'Others', 'Total']


In [15]:
temp_dict = {}
for row in table_rows[1:]:
    environment = row.text.split("\n")[0]
    val = int(row.text.split("\n")[2])
    # print(val)
    if val > 0:
        temp_dict[environment] = val

temp_dict

{'Environmental': 3, 'Sediment': 1, 'Others': 1, 'Total': 5}

# Join data

In [16]:
df_archaea_name_formatting = pd.read_csv(PATH_DF_URLS_ARCHAEA)
df_archaea_name_formatting.tail()

,name,url,formatted_name_gcm,formatted_name_ncbi,url_gcm_search,url_gcm_isolation_src
1232,uncultured_Nitrososphaera_sp.,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_Nitrososphaera_sp./,uncultured Nitrososphaera,uncultured Nitrososphaera sp.,https://gcm.wdcm.org/search?search=uncultured Nitrososphaera,https://gcm.wdcm.org/search?search=uncultured Nitrososphaera&list=isolate
1233,uncultured_archaeon_A07HB70,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HB70/,uncultured archaeon A07HB70,uncultured archaeon A07HB70,https://gcm.wdcm.org/search?search=uncultured archaeon A07HB70,https://gcm.wdcm.org/search?search=uncultured archaeon A07HB70&list=isolate
1234,uncultured_archaeon_A07HN63,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HN63/,uncultured archaeon A07HN63,uncultured archaeon A07HN63,https://gcm.wdcm.org/search?search=uncultured archaeon A07HN63,https://gcm.wdcm.org/search?search=uncultured archaeon A07HN63&list=isolate
1235,uncultured_archaeon_A07HR60,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HR60/,uncultured archaeon A07HR60,uncultured archaeon A07HR60,https://gcm.wdcm.org/search?search=uncultured archaeon A07HR60,https://gcm.wdcm.org/search?search=uncultured archaeon A07HR60&list=isolate
1236,uncultured_archaeon_A07HR67,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HR67/,uncultured archaeon A07HR67,uncultured archaeon A07HR67,https://gcm.wdcm.org/search?search=uncultured archaeon A07HR67,https://gcm.wdcm.org/search?search=uncultured archaeon A07HR67&list=isolate


In [24]:
dict_list = []
for index, row in df_archaea_name_formatting.loc[:5].iterrows():
    # print(index, row[['name', 'url_gcm_isolation_src']])
    driver.get(row['url_gcm_isolation_src'])
    table_rows = driver.find_elements("css selector", "div.fd-isolate table tr")
    found = False
    dict_organism = {}
    dict_organism['name'] = df_archaea_name_formatting.loc[index, 'name']
    dict_organism['url_refseq'] = df_archaea_name_formatting.loc[index, 'url']
    dict_organism['url_gcm_isolation_src'] = df_archaea_name_formatting.loc[index, 'url_gcm_isolation_src']
    
    for row in table_rows[1:]:
        environment = row.text.split("\n")[0]
        val = int(row.text.split("\n")[2])
        if val > 0:
            found = True
            dict_organism[environment] = val

    if found:
        dict_list.append(dict_organism)
    print(dict_organism)
print(dict_list)

with open(PATH_DESTINY_LIFESTYLE_ARCHAEA, 'a') as f:
    json.dump(dict_list, f, indent=4)


{'name': 'ANME-2_cluster_archaeon_HR1', 'url_refseq': 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/', 'url_gcm_isolation_src': 'https://gcm.wdcm.org/search?search=ANME-2 cluster archaeon HR1&list=isolate'}
{'name': 'Acidianus_ambivalens', 'url_refseq': 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_ambivalens/', 'url_gcm_isolation_src': 'https://gcm.wdcm.org/search?search=Acidianus ambivalens&list=isolate', 'Sediment': 1, 'Water': 1, 'Total': 2}
{'name': 'Acidianus_brierleyi', 'url_refseq': 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_brierleyi/', 'url_gcm_isolation_src': 'https://gcm.wdcm.org/search?search=Acidianus brierleyi&list=isolate', 'Environmental': 3, 'Sediment': 1, 'Others': 1, 'Total': 5}
{'name': 'Acidianus_hospitalis', 'url_refseq': 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_hospitalis/', 'url_gcm_isolation_src': 'https://gcm.wdcm.org/search?search=Acidianus hospitalis&list=isolate'}

In [26]:
df_archaea_name_formatting.loc[:]

,name,url,formatted_name_gcm,formatted_name_ncbi,url_gcm_search,url_gcm_isolation_src
0,ANME-2_cluster_archaeon_HR1,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/,ANME-2 cluster archaeon HR1,ANME-2 cluster archaeon HR1,https://gcm.wdcm.org/search?search=ANME-2 cluster archaeon HR1,https://gcm.wdcm.org/search?search=ANME-2 cluster archaeon HR1&list=isolate
1,Acidianus_ambivalens,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_ambivalens/,Acidianus ambivalens,Acidianus ambivalens,https://gcm.wdcm.org/search?search=Acidianus ambivalens,https://gcm.wdcm.org/search?search=Acidianus ambivalens&list=isolate
2,Acidianus_brierleyi,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_brierleyi/,Acidianus brierleyi,Acidianus brierleyi,https://gcm.wdcm.org/search?search=Acidianus brierleyi,https://gcm.wdcm.org/search?search=Acidianus brierleyi&list=isolate
3,Acidianus_hospitalis,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_hospitalis/,Acidianus hospitalis,Acidianus hospitalis,https://gcm.wdcm.org/search?search=Acidianus hospitalis,https://gcm.wdcm.org/search?search=Acidianus hospitalis&list=isolate
4,Acidianus_infernus,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_infernus/,Acidianus infernus,Acidianus infernus,https://gcm.wdcm.org/search?search=Acidianus infernus,https://gcm.wdcm.org/search?search=Acidianus infernus&list=isolate
...,...,...,...,...,...,...
1232,uncultured_Nitrososphaera_sp.,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_Nitrososphaera_sp./,uncultured Nitrososphaera,uncultured Nitrososphaera sp.,https://gcm.wdcm.org/search?search=uncultured Nitrososphaera,https://gcm.wdcm.org/search?search=uncultured Nitrososphaera&list=isolate
1233,uncultured_archaeon_A07HB70,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HB70/,uncultured archaeon A07HB70,uncultured archaeon A07HB70,https://gcm.wdcm.org/search?search=uncultured archaeon A07HB70,https://gcm.wdcm.org/search?search=uncultured archaeon A07HB70&list=isolate
1234,uncultured_archaeon_A07HN63,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HN63/,uncultured archaeon A07HN63,uncultured archaeon A07HN63,https://gcm.wdcm.org/search?search=uncultured archaeon A07HN63,https://gcm.wdcm.org/search?search=uncultured archaeon A07HN63&list=isolate
1235,uncultured_archaeon_A07HR60,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HR60/,uncultured archaeon A07HR60,uncultured archaeon A07HR60,https://gcm.wdcm.org/search?search=uncultured archaeon A07HR60,https://gcm.wdcm.org/search?search=uncultured archaeon A07HR60&list=isolate


# Bacteria

In [11]:
df_bacteria_name_formatting = pd.read_csv(PATH_DF_URLS_BACTERIA)
df_bacteria_name_formatting

,name,url,formatted_name_gcm,formatted_name_ncbi,url_gcm_search,url_gcm_isolation_src
0,Abditibacterium_utsteinense,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Abditibacterium_utsteinense/,Abditibacterium utsteinense,Abditibacterium utsteinense,https://gcm.wdcm.org/search?search=Abditibacterium utsteinense,https://gcm.wdcm.org/search?search=Abditibacterium utsteinense&list=isolate
1,Abiotrophia_defectiva,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Abiotrophia_defectiva/,Abiotrophia defectiva,Abiotrophia defectiva,https://gcm.wdcm.org/search?search=Abiotrophia defectiva,https://gcm.wdcm.org/search?search=Abiotrophia defectiva&list=isolate
2,Abiotrophia_sp.,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Abiotrophia_sp./,Abiotrophia,Abiotrophia sp.,https://gcm.wdcm.org/search?search=Abiotrophia,https://gcm.wdcm.org/search?search=Abiotrophia&list=isolate
3,Abiotrophia_sp._HMSC24B09,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Abiotrophia_sp._HMSC24B09/,Abiotrophia,Abiotrophia sp. HMSC24B09,https://gcm.wdcm.org/search?search=Abiotrophia,https://gcm.wdcm.org/search?search=Abiotrophia&list=isolate
4,Absicoccus_porci,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Absicoccus_porci/,Absicoccus porci,Absicoccus porci,https://gcm.wdcm.org/search?search=Absicoccus porci,https://gcm.wdcm.org/search?search=Absicoccus porci&list=isolate
...,...,...,...,...,...,...
52904,zeta_proteobacterium_SCGC_AB-137-J06,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/zeta_proteobacterium_SCGC_AB-137-J06/,zeta proteobacterium SCGC AB-137-J06,zeta proteobacterium SCGC AB-137-J06,https://gcm.wdcm.org/search?search=zeta proteobacterium SCGC AB-137-J06,https://gcm.wdcm.org/search?search=zeta proteobacterium SCGC AB-137-J06&list=isolate
52905,zeta_proteobacterium_SCGC_AB-602-C20,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/zeta_proteobacterium_SCGC_AB-602-C20/,zeta proteobacterium SCGC AB-602-C20,zeta proteobacterium SCGC AB-602-C20,https://gcm.wdcm.org/search?search=zeta proteobacterium SCGC AB-602-C20,https://gcm.wdcm.org/search?search=zeta proteobacterium SCGC AB-602-C20&list=isolate
52906,zeta_proteobacterium_SCGC_AB-602-E04,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/zeta_proteobacterium_SCGC_AB-602-E04/,zeta proteobacterium SCGC AB-602-E04,zeta proteobacterium SCGC AB-602-E04,https://gcm.wdcm.org/search?search=zeta proteobacterium SCGC AB-602-E04,https://gcm.wdcm.org/search?search=zeta proteobacterium SCGC AB-602-E04&list=isolate
52907,zeta_proteobacterium_SCGC_AB-604-B04,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/zeta_proteobacterium_SCGC_AB-604-B04/,zeta proteobacterium SCGC AB-604-B04,zeta proteobacterium SCGC AB-604-B04,https://gcm.wdcm.org/search?search=zeta proteobacterium SCGC AB-604-B04,https://gcm.wdcm.org/search?search=zeta proteobacterium SCGC AB-604-B04&list=isolate


In [12]:
df_bacteria_name_formatting[ df_bacteria_name_formatting['url'] == 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Acinetobacter_soli/' ]

,name,url,formatted_name_gcm,formatted_name_ncbi,url_gcm_search,url_gcm_isolation_src
543,Acinetobacter_soli,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Acinetobacter_soli/,Acinetobacter soli,Acinetobacter soli,https://gcm.wdcm.org/search?search=Acinetobacter soli,https://gcm.wdcm.org/search?search=Acinetobacter soli&list=isolate


In [13]:
df_bacteria_name_formatting[542:545]

,name,url,formatted_name_gcm,formatted_name_ncbi,url_gcm_search,url_gcm_isolation_src
542,Acinetobacter_silvestris,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Acinetobacter_silvestris/,Acinetobacter silvestris,Acinetobacter silvestris,https://gcm.wdcm.org/search?search=Acinetobacter silvestris,https://gcm.wdcm.org/search?search=Acinetobacter silvestris&list=isolate
543,Acinetobacter_soli,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Acinetobacter_soli/,Acinetobacter soli,Acinetobacter soli,https://gcm.wdcm.org/search?search=Acinetobacter soli,https://gcm.wdcm.org/search?search=Acinetobacter soli&list=isolate
544,Acinetobacter_sp.,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Acinetobacter_sp./,Acinetobacter,Acinetobacter sp.,https://gcm.wdcm.org/search?search=Acinetobacter,https://gcm.wdcm.org/search?search=Acinetobacter&list=isolate
